# Preprocessing des Referenzdatensatzes

1. Anonymisierung der @user-Erwähnungen zu "@user
2. Duplikate entfernen (bei Paaren sehr ähnlicher Tweets einen entfernen)
3. Mit den verschiedenen Annotationsebenen- und detaills separat speichern

## 1. Anonymisierung der @user-Erwähnungen

In [1]:
import re

def anonym_atuser(tweet):
    tweet_anonym = re.sub('@[^@ ]+?@', '@user@', tweet)
    tweet_anonym = re.sub('@[^@ ]+? ', '@user ', tweet_anonym)
    tweet_anonym = re.sub('@[^@ ]+?$', '@user', tweet_anonym)
    return tweet_anonym

# von '@rspctfl@houelle_beck @ergroovt @ThomasMichael71 @ksemann2 @DrKassandraPari Dann hättest Du nichts dagegen, wenn ein Lehrer ein Zeichen d PKK im Unterricht trägt? Oder die Flagge Israels? Oder ein Anstecker der AfD? @Innenwelttramp'
# zu '@user@user @user @user @user @user Dann hättest Du nichts dagegen, wenn ein Lehrer ein Zeichen d PKK im Unterricht trägt? Oder die Flagge Israels? Oder ein Anstecker der AfD? @user'

## 2. Duplikate entfernen

In [30]:
# Jaccard-Index-Funktionen zur Berechnung der Ähnlichkeit

def make_bag(string1):
    """ String als Multiset speichern; 
        In Form eines dicts: jedes vorkommende Unigram als Key, die Anzahl der Nutzungen als Value
        Als zusätzlichen Wert auch die Länge des Strings speichern
    """
    stringbag = dict()
    stringset = set(string1)
    for ch1 in stringset:
        stringbag[ch1] = string1.count(ch1)
    stringbag["len"] = len(string1)
    stringbag["tok"] = set()
    stringbag["tok"] = set(stringbag.keys()) - {"len", "tok"}
    return stringbag

def jaccard(set1, set2):
    """Jaccard-Index von zwei Sets (von zwei Strings) berechnen.
       Max.-Wert: 1 (Gleichheit)
    """
    jaccard_index = len(set1 & set2) / len(set1 | set2)
    return jaccard_index

def jaccard_multisets(bag1, bag2):
    """Den Jaccard-Index für zwei Multisets (von zwei Strings) berechnen.
       Max.-Wert: 0.5
    """
    # Schnittmenge bauen, die Gesamtlänge speichern
    schnitt_len = 0
    schnitt = bag1["tok"] & bag2["tok"]
    for gram in schnitt:
        l = min(bag1[gram],bag2[gram])
        schnitt_len += l

    # Länge der Vereinigung ermitteln
    vereinigung_len = bag1["len"] + bag2["len"]
    
    jaccard_index = schnitt_len / vereinigung_len
    return jaccard_index

In [29]:
tweet1 = "@Karl_Lauterbach Besser ein Amateur der lernfähig ist, als das verlauste Pack der abgefuckten SPD - SCHMAROTZER, PÄDOPHILE und DENUNZIANTEN !!!"
tweet2 = "@ThomasOppermann SPD - SCHMAROTZER,PÄDOPHILE UND DENUNZIANTEN   oder   SCHEINHEILGSTE PARTEI DEUTSCHLANDS !!!"
tweet3 = ""
tweet4 = "@spdde @hubertus_heil SPD - SCHMAROTZER, PÄDOPHILE UND DENUNZIANTEN"
tweet5 = "SPD - SCHMAROTZER, PÄDOPHILE UND DENUNZIANTEN"
tweet8 = "@user - SCHMAROTZER, PÄDOPHILE UND DENUNZIANTEN"
tweet6 = "Deutsche Medien, Halbwahrheiten und einseitige Betrachtung, wie bei allen vom Staat finanzierten 'billigen' Propagandainstitutionen 😜"
tweet7 = "Bevor sich PL an Angies Krimigranten-Quoten beteiligen wird, verlassen sie auch die EU u. Konzerne könnten ihren Mist ins Baltikum fliegen 😜"

#jind = jaccard_multisets(make_bag(tweet6), make_bag(tweet7))
#print(jind)

jacc = jaccard(set(tweet6), set(tweet7))
print(jacc)

0.6341463414634146


In [82]:
def calculate_bags(corpus):
    """Multisets (bags) für alle Tweets in einem Korpus berechen.
    Input: Korpus
    Output: Liste von Multisets (Index in der Liste == Index in der Korpusliste)
    """
    bags = [make_bag(tweet[1]) for tweet in corpus]
    return bags

def calculate_sets(corpus):
    """Set für alle Tweets in einem Korpus berechnen; als Liste der Sets speichern.
    """
    sets = [set(tweet[1]) for tweet in corpus]
    return sets

def collect_duplicates(corpus, sets, simfunc, cutoff):
    """ Duplikate sammeln
    Input: Korpus, Liste von (Multi)Sets für alle Tweets im Korpus, Vergleichsfunktion, Ähnlichkeitsgrenzwert
    Output: Liste potentieller Duplikate (als Tupel der Korpuseinträge)
    """
    duplicates = []
    for i in range(len(corpus)):
        # if i % 100 == 0: print("i",i) # Zur Zeitmessung
        for j in range(len(corpus)):
            # Kein Vergleich mit sich selbst oder bereits in die andere Richtung verglichener Paare
            if i >= j: continue
            else:
                jacc = simfunc(sets[i],sets[j])
                if jacc >= cutoff: # z.B. 0.47 für multisets
                    duplicates.append((corpus[i],corpus[j]))
    return duplicates

In [75]:
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train.txt", mode="r", encoding="utf-8") as in_train:
    train = in_train.readlines()

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test.txt", mode="r", encoding="utf-8") as in_test:
    test = in_test.readlines()

total = train[1:] + test[1:] # ohne Erklärungszeile
total = [entry.strip().split("\t") for entry in total]


# @user-Erwähnungen anonymisieren
total_anonym = [(entry[0],anonym_atuser(entry[1]),entry[2],entry[3]) for entry in total]

# Sets aller Tweets berechnen
tweet_sets = calculate_sets(total_anonym)

#### Jaccard-Index berechnen

In [80]:
bags = calculate_bags(total_anonym) # Multimenge für alle Tweets berechnen

In [38]:
# Jaccard-Index berechnen
dups_multisets = collect_duplicates(total_anonym, bags, jaccard_multisets, 0.47)
# Berechnung (anfangs): ca. 30 Sekunden/200 Tweets --> insg. ca 70 Minuten
# Aber: Wird mit der Zeit schneller (da nach und nach bereits alle Vergleiche in eine Richtung bereits geschehen) --> insg.: 42m 9.9s

i 0
i 100
i 200
i 300
i 400
i 500
i 600
i 700
i 800
i 900
i 1000
i 1100
i 1200
i 1300
i 1400
i 1500
i 1600
i 1700
i 1800
i 1900
i 2000
i 2100
i 2200
i 2300
i 2400
i 2500
i 2600
i 2700
i 2800
i 2900
i 3000
i 3100
i 3200
i 3300
i 3400
i 3500
i 3600
i 3700
i 3800
i 3900
i 4000
i 4100
i 4200
i 4300
i 4400
i 4500
i 4600
i 4700
i 4800
i 4900
i 5000
i 5100
i 5200
i 5300
i 5400
i 5500
i 5600
i 5700
i 5800
i 5900
i 6000
i 6100
i 6200
i 6300
i 6400
i 6500
i 6600
i 6700
i 6800
i 6900
i 7000
i 7100
i 7200
i 7300
i 7400
i 7500
i 7600
i 7700
i 7800
i 7900
i 8000
i 8100
i 8200
i 8300
i 8400
i 8500
i 8600
i 8700
i 8800
i 8900
i 9000
i 9100
i 9200
i 9300
i 9400
i 9500
i 9600
i 9700
i 9800
i 9900
i 10000
i 10100
i 10200
i 10300
i 10400
i 10500
i 10600
i 10700
i 10800
i 10900
i 11000
i 11100
i 11200
i 11300
i 11400
i 11500
i 11600
i 11700
i 11800
i 11900
i 12000
i 12100
i 12200
i 12300
i 12400
i 12500
i 12600
i 12700
i 12800
i 12900
i 13000
i 13100
i 13200
i 13300
i 13400
i 13500
i 13600
i 13700
i 13800


In [42]:
#import json
#dups_json = json.dumps(dups_multisets)
#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\Duplikat_IDs.json", mode="w", encoding="utf-8") as out_dups:
#    out_dups.write(dups_json)

In [67]:
# Duplikat-Cluster bestimmen, um Kettenvergleiche zu vermeiden
# --> falls ID1 ähnlich ID2: zusammen in ein Set, und jede weitere ID, die einer der beiden IDs ähnlich ist dazu

cluster = []
ID_cluster_ref = dict()
for duplicate_tup in dups_multisets:
    tweet1, tweet2 = duplicate_tup
    ID1, ID2 = tweet1[0], tweet2[0]
    # 1. weder ID1 noch ID2 vorhanden
    if (ID1 not in ID_cluster_ref) and (ID2 not in ID_cluster_ref):
        cluster_num = len(cluster)
        ID_cluster_ref[ID1], ID_cluster_ref[ID2] = cluster_num, cluster_num
        cluster.append({tweet1, tweet2})
    # 2. eine von beiden vorhanden --> beide IDs ins vorhandene Cluster integrieren
    elif ID1 in ID_cluster_ref:
        cluster_num = ID_cluster_ref[ID1]
        ID_cluster_ref[ID2] = cluster_num
        cluster[cluster_num].add(tweet2)
    elif ID2 in ID_cluster_ref:
        cluster_num = ID_cluster_ref[ID2]
        ID_cluster_ref[ID1] = cluster_num
        cluster[cluster_num].add(tweet1)
    # 3. Beide bereits vorhanden
    else:
        assert ID_cluster_ref[ID1] == ID_cluster_ref[ID2]

# schwarze Liste zu löschender IDs
schwarzeListe = []
for gruppe in cluster[2:]:
    grp = list(gruppe)
    for tweet in grp[1:]:
        schwarzeListe.append(tweet[0])


In [71]:
print(len(ID_cluster_ref)) # 321 Tweets, die einander in irgendeiner Konstellation ähnlich sind
print(len(cluster)) # 100 Cluster
print(cluster[0]) 
print(len(schwarzeListe)) # 223 zu löschende Tweets

# Cluster 0: vergessene erste Zeile (Erklärung) --> TODO nochmal laufen lassen, json neu speichern
# Cluster 1: Ähnlichkeit durch wiederholte @user Erwähnungen, Text selbst nicht ähnlich
# tatsächlich ähnlich, nur einen der Tweets behalten (Zufall): Clusters 2 - 8 
# Cluster 4: 04110200 und zwei weitere HASOC2020 Tweets: "https:/…" als Rest -> mit generischem Link ersetzten

321
100
{('corpus_id', 'tweet', 'binarylabel', 'finelabel')}
223


In [84]:
# Ohne die Duplikate in neue Dateien schreiben (nicht mehr anonymisiert)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_OD.txt", mode="w", encoding="utf-8") as out_train:
    for tweet in train:
        if tweet[0] not in schwarzeListe:
            out_train.write(tweet)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_OD.txt", mode="w", encoding="utf-8") as out_test:
    for tweet in test:
        if tweet[0] not in schwarzeListe:
            out_test.write(tweet)

## 3. Annotationen formatieren und speichern

- in unterschiedlicher Detailgenauigkeit speichern:
    - VVH Ja/Nein, VVH-Allg Ja/Nein, VVH-NS Ja/Nein
    - Gruppe Ja/Nein, Welches Gruppenmerkmal/Keine Gruppe, Welche Gruppe genau/Keine Gruppe 
    - Handlung VVH-Allg Ja/Nein, Welche Handlung genau/Keine Handlung
    - Handlung VVH-NS Ja/Nein, Welche Handlung genau/Keine Handlung

- gleichzeitig @user-Erwähnungen anonymisieren


In [93]:
import json

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE_anno_korrigiert.json", mode="r", encoding="utf-8") as in_hate_train:
    hate_train = in_hate_train.read()
    hate_train_anno = json.loads(hate_train)
    for entry in hate_train_anno:
        entry["data"] = anonym_atuser(entry["data"])


with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE_anno_korrigiert.json", mode="r", encoding="utf-8") as in_hate_test:
    hate_test = in_hate_test.read()
    hate_test_anno = json.loads(hate_test)
    for entry in hate_test_anno:
        entry["data"] = anonym_atuser(entry["data"])

# Beispiel-Eintrag: {"id": "01222380", "data": "den #Zentralrat der #Muslime sollte man komplett rausschmei\u00dfen, er hat hier nichts zu suchen @aktuelleStunde #WDR", "label": ["KEINE"], "tag1": "NEG", "tag2": "HATE"}

# Verallgemeinerte Annotation

# 1. VVH Ja/Nein:
def transform_anno_vvh(corp):
    new_anno = []
    for entry in corp:
        if "KEINE" in entry["label"]:
            entry["label"] = "KEINE"
        else:
            entry["label"] = "VVH"
        new_anno.append(entry)
    return new_anno

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Train_HATE_VVH.txt", mode="w", encoding="utf-8") as out_vvh_train:
    for i in transform_anno_vvh(hate_train_anno):
        if i["id"] not in schwarzeListe:
            out_vvh_train.write("\t".join((i["id"], i["data"], i["label"]))+"\n")

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Test_HATE_VVH.txt", mode="w", encoding="utf-8") as out_vvh_test:
    for i in transform_anno_vvh(hate_test_anno):
        if i["id"] not in schwarzeListe:
            out_vvh_test.write("\t".join((i["id"], i["data"], i["label"]))+"\n")



# 2.1   Gruppe Ja/Nein
def transform_anno_gruppe(corp):
    gruppe = {"Nationalität", 'ethnische Herkunft / "Rasse"', "Religion / Weltanschauung",
                    "Politische Einstellung", "Geschlecht", "Anderes Merkmal"}
    new_anno = []
    for entry in corp:
        if gruppe & set(entry["label"]) != set():
            entry["label"] = "Gruppe"
        else:
            entry["label"] = "KeineGruppe"
        new_anno.append(entry)
    return new_anno

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Train_HATE_Gruppe.txt", mode="w", encoding="utf-8") as out_gr_train:
    for i in transform_anno_gruppe(hate_train_anno):
        if i["id"] not in schwarzeListe:
            out_gr_train.write("\t".join((i["id"], i["data"], i["label"]))+"\n")

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Test_HATE_Gruppe.txt", mode="w", encoding="utf-8") as out_gr_test:
    for i in transform_anno_gruppe(hate_test_anno):
        if i["id"] not in schwarzeListe:
            out_gr_test.write("\t".join((i["id"], i["data"], i["label"]))+"\n")


# 2.2   Welche Gruppe / Keine Gruppe
def transform_anno_gruppe_det(corp):
    gruppe = {"Nationalität", 'ethnische Herkunft / "Rasse"', "Religion / Weltanschauung",
                    "Politische Einstellung", "Geschlecht", "Anderes Merkmal"}
    new_anno = []
    for entry in corp:
        if gruppe & set(entry["label"]) != set():
            entry["label"] = gruppe & set(entry["label"])
        else:
            entry["label"] = "KeineGruppe"
        new_anno.append(entry)
    return new_anno

#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Train_HATE_GruppeDetail.txt", mode="w", encoding="utf-8") as out_grd_train:
#    for i in transform_anno_gruppe_det(hate_train_anno):
#        out_grd_train.write()

#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Test_HATE_GruppeDetail.txt", mode="w", encoding="utf-8") as out_grd_test:
#    for i in transform_anno_gruppe_det(hate_test_anno):
#        out_grd_test.write()


# 3.1   Handlung VVH-Allg Ja/Nein
def transform_anno_handlung(corp):
    handlung = {"Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"}
    new_anno = []
    for entry in corp:
        if handlung & set(entry["label"]) != set():
            entry["label"] = "HandlungVVH"
        else:
            entry["label"] = "KeineHandlung"
        new_anno.append(entry)
    return new_anno

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Train_HATE_Handlung.txt", mode="w", encoding="utf-8") as out_handl_train:
    for i in transform_anno_handlung(hate_train_anno):
        if i["id"] not in schwarzeListe:
            out_handl_train.write("\t".join((i["id"], i["data"], i["label"]))+"\n")

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Test_HATE_Handlung.txt", mode="w", encoding="utf-8") as out_handl_test:
    for i in transform_anno_handlung(hate_test_anno):
        if i["id"] not in schwarzeListe:
            out_handl_test.write("\t".join((i["id"], i["data"], i["label"]))+"\n")



# 3.2   Welche Handlung VVH-Allg / Keine Handlung 
def transform_anno_handlung_det(corp):
    handlung = {"Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"}
    new_anno = []
    for entry in corp:
        if handlung & entry["label"] != set():
            entry["label"] = handlung & set(entry["label"])
        else:
            entry["label"] = "KeineHandlung"
        new_anno.append(entry)
    return new_anno

#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Train_HATE_HandlungDetail.txt", mode="w", encoding="utf-8") as out_handld_train:
#    for i in transform_anno_handlung_det(hate_train_anno):
#        out_handld_train.write()

#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Test_HATE_HandlungDetail.txt", mode="w", encoding="utf-8") as out_handld_test:
#    for i in transform_anno_handlung_det(hate_test_anno):
#        out_handld_test.write()



In [94]:
txtPfade = ["..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Train_HATE_VVH.txt",
            "..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Test_HATE_VVH.txt",
            "..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Train_HATE_Gruppe.txt",
            "..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Test_HATE_Gruppe.txt",
            "..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Train_HATE_Handlung.txt",
            "..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Test_HATE_Handlung.txt",
            ]

import json

for datei in txtPfade:
    with open(datei, mode="r", encoding="utf8") as in_txt:
        inhalt = in_txt.readlines()
        sep_inhalt = [eintrag.strip().split("\t") for eintrag in inhalt]
        json_inhalt = json.dumps(sep_inhalt)
        with open(datei.strip("txt")+"json", mode="w", encoding="utf-8") as out_json:
            out_json.write(json_inhalt)